# Librerias

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import utils as u
import re
import geopandas as gpd
import ast

pd.options.display.max_columns = None

In [6]:
''' 
Lectura de datos de restaurantes de Madrid.
'''
restaurantes = pd.read_csv('../data/processed/restaurantes.csv')
restaurantes.head(1)

,nombre_restaurante,lat,lon,place_id,dine_in,direccion,price_level,rating,reservable,serves_beer,serves_breakfast,serves_brunch,serves_dinner,serves_lunch,serves_vegetarian_food,serves_wine,takeout,user_ratings_total,delivery,cod_distrito,cod_barrio,price_level_mean,rating_mean,user_ratings_mean,apt_municip_mayores,asocia_sec_1,asocia_cult_casas_reg,asocia_mujeres,asocia_vecinales,anio_medio_constr_vivendas,bibliotecas_comunidad,bibliotecas_municipal,bibliotecas_comunidad_public,bibliotecas_municipal_public,campus_futb_11,centro_ninos_3_12_anios,centro_municipal_mayor,cmsc,aspa,caf,cai,cad_y_ccad,centro_dia_alzheimer_fisicos,centro_serv_social,centro_deport_municipal,centro_pers_sin_hogar,centro_espacio_cultural,colegio_publ_infatil_primaria,dur_media_credito_viviendas,edad_media_poblacion,votos_blancos_elec_municipal,abstenciones_elec_municipal,censo_elec_municipal,votos_ciudadanos_elec_municipal,votos_masmadrid_elec_municipal,votos_masverdeequo_elec_municipal,votos_podemos_elec_municipal,votos_vox_elec_municipal,votos_candidaturas_elec_municipal,votos_pp_elec_municipal,votos_psoe_elec_municipal,escuelas_inf_municipal,espacio_igualdad,espacio_adolecentes_ocio,esperanza_vida_hombres,esperanza_vida_mujeres,etapa_educativa_ninas,etapa_educativa_ninos,fundacion_sec_2,hogar_hombre_mas_65,hogar_mujer_mas_65,hogar_monoparental_hombre,hogar_monoparental_mujer,instalaciones_deportivas_basc,inst_publico_secundaria,mercado_municipales,num_habitantes,num_inmuebles_residencial,num_locales_alta_abiertos,num_locales_alta_cerrados,num_locales_alta_obra,num_locales_alta_vivienda,num_locales_alta_tot,parados,parados_hombre,parados_mujer,pension_media_mensual_mujeres,pension_media_mensual_hombres,personas_espanolas,personas_espanolas_hombre,personas_espanolas_mujer,personas_extranjera,personas_extranjera_hombre,personas_extranjera_mujer,piscina_cubierta,piscina_verano,pista_atletismo,poblacion_hombre,poblacion_mujer,poblacion_0_14,poblacion_15_29,poblacion_30_44,poblacion_45_64,poblacion_65_79,poblacion_65_mas,poblacion_80_mas,poblacion_densidad,poblacion_etapa_educativa_0_2,poblacion_etapa_educativa_12_15,poblacion_etapa_educativa_3_5,poblacion_etapa_educativa_6_11,poblacion_etapa_educativa,poblacion_etapa_educativa_0_2_mujer,poblacion_etapa_educativa_12_15_mujer,poblacion_etapa_educativa_3_5_mujer,poblacion_etapa_educativa_6_11_mujer,poblacion_etapa_educativa_0_2_hombre,poblacion_etapa_educativa_12_15_hombre,poblacion_etapa_educativa_3_5_hombre,poblacion_etapa_educativa_6_11_hombre,pobl_25_mas_est_superiores,pobl_25_mas_no_letrados,pobl_25_mas_bachiller_eso_fp1,pobl_25_mas_bachillersup_fp2,pobl_25_mas_estudio_desconocido,pobl_25_mas_primaria_incompleta,pobl_25_titu_media,pct_evejecimiento,pct_sobrevejecimiento,posicion_ranking_vulnerabilidad,poblacion_bangladesh,poblacion_bolivia,poblacion_china,poblacion_ecuador,poblacion_filipinas,poblacion_francia,poblacion_italia,poblacion_paraguay,poblacion_peru,poblacion_rumania,poblacion_venezuela,poblacion_colombia,poblacion_francia.1,poblacion_honduras,pct_migrantes(personas_extranjeras_menos_UE/poblacion_total),pct_migrantes(poblacion_extranjeras_menos_UE/poblacion_total),renta_media_persona,renta_neta_anual_hogar,residencia_mayores,superficie,superficie_media_residencias,superficie_media_vivienda_transac,tamano_hogar_medio,pct_abs_paro,pct_abs_paro_hombre,pct_abs_paro_mujer,pct_bruta_natalidad,pct_crecimiento_demografico,pct_desempleo_hombre_16-24,pct_desempleo_hombre_25-44,pct_desempleo_hombre_45-64,pct_desempleo_mujer_16-24,pct_desempleo_mujer_25-44,pct_desempleo_mujer_45-64,tot_hogares,valor_catast_inmueble_residen,indice_vulnerabilidad_bienestar_social_igual,indice_vulnerabilidad_eco_empleo,indice_vulnerabilidad_edu_cultura,indice_vulnerabilidad_ambiente_urbano y movi,indice_vulnerabilidad_salud,indice_vulnerabilidad_territorial,indice_dependiencia(bebes y aldutos mayores/pob 16-64),tipo_cocina,Americana / Burgers,Asiática,China,Española,Fusión,Italiana,Japonesa,Latinoamericana,Mexicana,Otros,A

In [11]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.pipeline import Pipeline


X = restaurantes[['serves_breakfast', 
                  'parados', 
                  'dur_media_credito_viviendas', 
                  'poblacion_80_mas',
                  'poblacion_china',
                  'pct_crecimiento_demografico',
                  'rating_mean',
                  'poblacion_italia',
                  'user_ratings_mean',
                  'price_level']]
y = restaurantes['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipe = Pipeline([
    ('poly', PolynomialFeatures()),
    ('scaler', StandardScaler()),
    ('classifier', LinearRegression())])

linear_params = {
    'poly__degree':[1],
    'scaler':[MinMaxScaler(), StandardScaler()],
    'classifier': [LinearRegression()]
}

regularizacion_params = {
    'poly__degree':[1, 2, 3, 4, 5],
    'scaler': [MinMaxScaler(), StandardScaler()],
    'classifier': [Ridge(), Lasso()],
    'classifier__alpha': [0.25, 0,75, 0.80, 0.90, 1, 100]
}

elastic_param = {
    'poly__degree':[1, 2, 3, 4, 5],
    'scaler': [MinMaxScaler(), StandardScaler()],
    'classifier': [ElasticNet()],
    'classifier__alpha': [0.25, 0,75, 0.80, 0.90, 1, 100],
    'classifier__l1_ratio': [0.1, 0.25, 0.50, 0.75, 0.80, 1]
}

search_space = [
    linear_params,
    regularizacion_params,
    elastic_param
]

clf = GridSearchCV(estimator = pipe,
                  param_grid = search_space,
                  scoring='neg_mean_absolute_error',
                  cv = 10,
                  n_jobs=-1,
                  verbose=2)

clf.fit(X_train, y_train)

print(clf.best_estimator_)
print(clf.best_score_)
print(clf.best_params_)

Fitting 10 folds for each of 562 candidates, totalling 5620 fits
Pipeline(steps=[('poly', PolynomialFeatures()), ('scaler', MinMaxScaler()),
                ('classifier', Ridge(alpha=0.25))])
-5.081940554111285
{'classifier': Ridge(), 'classifier__alpha': 0.25, 'poly__degree': 2, 'scaler': MinMaxScaler()}


In [12]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score

best1 = clf.best_estimator_
predictions_best1 = best1.predict(X_test)

print("MAE test", mean_absolute_error(y_test, predictions_best1))
print("MAPE test", mean_absolute_percentage_error(y_test, predictions_best1))
print("MSE test", mean_squared_error(y_test, predictions_best1))
print("RMSE test", mean_squared_error(y_test, predictions_best1)**(1/2))
print("R2 score", r2_score(y_test, predictions_best1))

MAE test 5.209969983609404
MAPE test 0.41869564397943093
MSE test 44.72980831554107
RMSE test 6.688034712495224
R2 score 0.33632876074056617


In [13]:
restaurantes['y'].describe()

count    2544.000000
mean       25.129694
std         8.028776
min         0.693147
25%        20.834239
50%        26.477211
75%        30.782543
max        50.797956
Name: y, dtype: float64

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost

